# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch

In [2]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

     |████████████████████████████████| 4.0 MB 11.0 MB/s 
     |████████████████████████████████| 895 kB 23.8 MB/s 
     |████████████████████████████████| 596 kB 36.2 MB/s 
     |████████████████████████████████| 6.6 MB 32.0 MB/s 
     |████████████████████████████████| 77 kB 1.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/SMAI/Assignment3/

/content/drive/MyDrive/SMAI/Assignment3


In [4]:
# loading the dataset

train_df = pd.read_csv('q4_data/train.csv')
test_df = pd.read_csv('q4_data/test.csv')

#dataset understanding
train_df.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
20705,36a652a6b808096a,"According to Yeager's official bio, he is stil...",0,0,0,0,0,0
82884,ddc86228e9e48040,"""\n\nIf you have a reliable source that talks ...",0,0,0,0,0,0
149926,63b7188b3804fdc1,"Congrats! Just remember, it's quality, not qua...",0,0,0,0,0,0
92892,f85eb3efa425d70f,Userboxes \n\nI think that the actions of this...,0,0,0,0,0,0
57732,9a7ec25a0f769b6e,"I don't know what mental illnes you have, but ...",1,0,1,0,1,0


In [5]:
# let's see the total rows in train, test data and the numbers for the various categories
cols_target = ['toxic', 'insult', 'severe_toxic', 'identity_hate','threat','obscene']
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
print(train_df[cols_target].sum())

Total rows in test is 153164
Total rows in train is 159571
toxic            15294
insult            7877
severe_toxic      1595
identity_hate     1405
threat             478
obscene           8449
dtype: int64


In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [7]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [8]:
# train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)

(159571,) (153164,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [ ]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.load("https://tfhub.dev/google/elmo/3", trainable=True)

In [ ]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [ ]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
# #Obtain X_test and X_train 
# from tqdm import tqdm
# X_train_use = []
# for x in tqdm(X):
#   X_train_use.append(get_use_embedding(x))

X_test_use = []
for x in tqdm(test_X):
  X_test_use.append(get_use_embedding(x))

100%|██████████| 153164/153164 [28:35<00:00, 89.31it/s]


In [ ]:
# np.save("use_x_train", np.array(X_train_use))
np.save("use_x_test", np.array(X_test_use))

In [9]:
x_train_use = np.load('embeddings.npy')
x_test_use = np.load('use_x_test.npy')

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [10]:
#calculate the accuracies obtained, u can use sklearn metrics
y_train = train_df.drop(labels = ['id','comment_text'], axis=1)
y_test = test_df.drop(labels = ['id','comment_text'], axis=1)

In [11]:
y_text = pd.read_csv('q4_data/test.csv')
y_labels = pd.read_csv('q4_data/test_labels.csv')
y_text_labels = pd.merge(y_text, y_labels)
y_text_labels.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1


In [12]:
indexes = y_text_labels[ (y_text_labels['toxic'] == -1) & (y_text_labels['severe_toxic'] == -1) & (y_text_labels['identity_hate'] == -1) & (y_text_labels['obscene'] == -1) & (y_text_labels['threat'] == -1) & (y_text_labels['insult'] == -1) ].index
y_text_labels.drop(indexes,inplace=True)

In [13]:
y_text_labels.shape

(63978, 8)

In [14]:
y_labels.drop(indexes, inplace=True)

In [15]:
req_embedding_indexes = list(y_text_labels.index)
len(req_embedding_indexes)

63978

In [16]:
req_x_test_use = [x_test_use for i in req_embedding_indexes]

In [17]:
x_test_use = req_x_test_use

In [18]:
toxic_y_train = train_df.toxic
severe_toxic_y_train = train_df.severe_toxic
obscene_y_train = train_df.obscene
threat_y_train = train_df.threat
insult_y_train = train_df.insult
identity_hate_y_train = train_df.identity_hate

In [ ]:
# Naive Bayes classifier 
from sklearn.naive_bayes import GaussianNB

toxic_nb = GaussianNB()
toxic_nb.fit(x_train_use[:100], toxic_y_train[:100])
toxic_pred = toxic_nb.predict(x_test_use)

severe_toxic_nb = GaussianNB()
severe_toxic_nb.fit(x_train_use[:100], severe_toxic_y_train[:100])
severe_toxic_pred = severe_toxic_nb.predict(x_test_use)

obscene_nb = GaussianNB()
obscene_nb.fit(x_train_use[:100], obscene_y_train[:100])
obscene_pred = obscene_nb.predict(x_test_use)

threat_nb = GaussianNB()
threat_nb.fit(x_train_use[:100], threat_y_train[:100])
threat_pred = threat_nb.predict(x_test_use)

identity_hate_nb = GaussianNB()
identity_hate_nb.fit(x_train_use[:100], identity_hate_y_train[:100])
identity_hate_pred = identity_hate_nb.predict(x_test_use)

insult_nb = GaussianNB()
insult_nb.fit(x_train_use[:100], insult_y_train[:100])
insult_pred = insult_nb.predict(x_test_use)

In [ ]:
# Naive Bayes classifier 
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(x_train_use, y_train)
# predict
predictions = classifier.predict(x_test_use)
# accuracy
print("Accuracy = ",accuracy_score(y_labels,predictions))

In [20]:
predictions[0]

<1x6 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Column format>

In [ ]:
#Support Vector Machines

In [ ]:
# MLP
from keras.models import Sequential
from keras.layers import Dense

def create_model_single_class(input_dim, output_dim):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=input_dim, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(output_dim, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [1]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.ext import Keras

KERAS_PARAMS = dict(epochs=10, batch_size=100, verbose=0)

clf = BinaryRelevance(classifier=Keras(create_model_single_class, False, KERAS_PARAMS), require_dense=[True,True])
clf.fit(x_train_use, y_train)
result = clf.predict(x_test_use)

NameError: ignored

Which classifier of the three performed better, and why do you think so ?

Ans:

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [1]:
import math
learning_rate = 0.001
hidden_layers = 100

In [8]:
!pip3 install mlfromscratch

  Using cached mlfromscratch-0.0.4.tar.gz (36 kB)
  Using cached mlfromscratch-0.0.3.tar.gz (34 kB)
  Using cached mlfromscratch-0.0.2.tar.gz (34 kB)
  Using cached mlfromscratch-0.0.1.tar.gz (34 kB)
ERROR: Could not find a version that satisfies the requirement mlfromscratch (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4)
ERROR: No matching distribution found for mlfromscratch


In [5]:
from mlfromscratch.deep_learning.activation_functions import Sigmoid, Softmax
from mlfromscratch.deep_learning.loss_functions import CrossEntropy

ModuleNotFoundError: ignored

In [2]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed
def sigmoid(x):
    return(1/(1 + np.exp(-x)))

def softmax():

def loss(self, y, p):
    # Avoid division by zero
    p = np.clip(p, 1e-15, 1 - 1e-15)
    return - y * np.log(p) - (1 - y) * np.log(1 - p)

def initialise_weights(X, y):
  n_samples, n_features = X.shape
  _, n_outputs = y.shape
  # Hidden layer
  limit   = 1 / math.sqrt(n_features)
  W  = np.random.uniform(-limit, limit, (n_features, hidden_layers))
  w0 = np.zeros((1, hidden_layers))
  # Output layer
  limit   = 1 / math.sqrt(hidden_layers)
  V  = np.random.uniform(-limit, limit, (hidden_layers, n_outputs))
  v0 = np.zeros((1, n_outputs))
  return W, w0, V, v0

def forward_layer(X, W, w0, V, v0):
   # HIDDEN LAYER
    hidden_input = X.dot(W) + w0
    hidden_output = sigmoid(hidden_input)
    # OUTPUT LAYER
    output_layer_input = hidden_output.dot(V) + v0
    y_pred = softmax(output_layer_input)

  return y_pred


def backward_propogation():
  return 



In [ ]:
def train():
  

In [ ]:
#report accuracies
#here, you need to predict for each of the label as we have trained classifiers in such a way

def predict():




In [ ]:
#report accuracies on the test dataset 





Now, compare your implemented MLP with the sklearn MLP layer.

In [ ]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier



Which MLP performed better? 
Understand and Analyse.